# Sentiment Analysis Using BagOfWords 

In [1]:
import re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

In [2]:
#loading the data files
train = pd.read_csv('labeledTrainData.tsv',quoting=3,header=0,delimiter='\t')
test = pd.read_csv('testData.tsv',quoting=3,header=0,delimiter='\t')
data = pd.read_csv('unlabeledTrainData.tsv', quoting=3, header=0, delimiter='\t')

In [3]:
train.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [4]:
train.review[0]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

In [5]:
def clean_review(movie_review):
    #Removing HTML tags
    review = BeautifulSoup(movie_review, "lxml").get_text()
    
    #Removing non alphabetic words
    review = re.sub("[^a-zA-Z]"," ",review)
    return review.lower()

In [6]:
train.review = [clean_review(review) for review in train.review]
test.review = [clean_review(review) for review in test.review]
data.review = [clean_review(review) for review in data.review]

In [7]:
train.review[0]

u' with all this stuff going down at the moment with mj i ve started listening to his music  watching the odd documentary here and there  watched the wiz and watched moonwalker again  maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  some of it has subtle messages about mj s feeling towards the press and also the obvious message of drugs are bad m kay visually impressive but of course this is all about michael jackson so unless you remotely like mj in anyway then you are going to hate this and find it boring  some may call mj an egotist for consenting to the making of this movie but mj and most of his fans would say that he made it for the fans which if true is really nice of him the actual feature film bit when it finally starts is only on for    

In [8]:
bow_vec = CountVectorizer(max_features=75000, ngram_range=(1,2))

In [9]:
training_text = data.review.tolist()
for review in train.review:
    training_text.append(review)

In [10]:
%%time
bow_vec.fit(training_text)

CPU times: user 1min 28s, sys: 2.93 s, total: 1min 31s
Wall time: 1min 32s


CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=75000, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [12]:
del data

In [13]:
X_bow = bow_vec.transform(train.review).toarray()

In [14]:
X_train,X_val,y_train,y_val = train_test_split(X_bow, train.sentiment, test_size=0.3)

## Training An Ensemble of SGDClassifier, LogisticRegression & MultinomialNB classifiers to predict the sentiment of a review

In [36]:
%%time
bow_sgd = SGDClassifier(alpha=0.0004, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)
bow_sgd.fit(X_train, y_train)

CPU times: user 47.8 s, sys: 1min 28s, total: 2min 16s
Wall time: 2min 58s


In [37]:
print "Training score: %g, Cross-Validation Score: %g"%(bow_sgd.score(X_train,y_train),bow_sgd.score(X_val,y_val))

Training score: 0.967314, Cross-Validation Score: 0.8756


#### After Hyper-parameter tuning using the entire data to train the SGDClassifier

In [54]:
bow_sgd.fit(X_bow, train.sentiment)

SGDClassifier(alpha=0.0004, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

### Training LogisticRegression Classifier

In [43]:
%%time
bow_lr = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=0.05, fit_intercept=True,
                            intercept_scaling=1, class_weight=None, random_state=None, solver='liblinear',
                            max_iter=10, multi_class='ovr', verbose=0, warm_start=False, n_jobs=1)
bow_lr.fit(X_train, y_train)

CPU times: user 21.6 s, sys: 38.7 s, total: 1min
Wall time: 1min 20s


In [44]:
print "Training score: %g, Cross-Validation Score: %g"%(bow_lr.score(X_train,y_train),bow_lr.score(X_val,y_val))

Training score: 0.992686, Cross-Validation Score: 0.892


In [55]:
bow_lr.fit(X_bow, train.sentiment)

LogisticRegression(C=0.05, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=10, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

### Training MultinomialNB Classifer

In [52]:
%%time
bow_mnb = MultinomialNB(alpha=1.0, fit_prior=True, class_prior=None)
bow_mnb.fit(X_train, y_train)

CPU times: user 15.9 s, sys: 51.8 s, total: 1min 7s
Wall time: 1min 3s


In [53]:
print "Training score: %g, Cross-Validation Score: %g"%(bow_mnb.score(X_train,y_train),bow_mnb.score(X_val,y_val))

Training score: 0.9184, Cross-Validation Score: 0.8628


In [56]:
bow_mnb.fit(X_bow, train.sentiment)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [57]:
X_test = bow_vec.transform(test.review)

### Making an Ensemble of the above classifiers

In [60]:
sgd_pred = bow_sgd.predict_proba(X_test)
lr_pred = bow_lr.predict_proba(X_test)
mnb_pred = bow_mnb.predict_proba(X_test)

In [61]:
result = np.argmax(sgd_pred + lr_pred + mnb_pred, axis=1)

In [63]:
output = pd.DataFrame(data={"id":test["id"],"sentiment":result})

In [64]:
output.to_csv("BagOfWords.csv",index=False,quoting=3)

Got 0.89504 on Kaggle